In [1]:
import pandas as pd 
import numpy as np


# Data PreProcessing

In [2]:
df=pd.read_excel("Training set.xlsx",sheet_name="User-model")
knowledges=pd.read_excel("Knowledge.xlsx",sheet_name="Sheet1")
knowledges=knowledges.rename(columns={"final_answer": "Final_answer","system_question1": "System_question1", "system_queston3": "System_question3", "system_question2": "System_question2"})

knowledge=knowledges.copy()

#Encoding processing

In [3]:
Final_df=df.copy()


In [4]:
from sklearn.preprocessing import LabelEncoder
age_encoder= LabelEncoder()
Final_df["Age"]=age_encoder.fit_transform(Final_df["Age"])

In [5]:
from sklearn.preprocessing import LabelEncoder
school_degree_encoder= LabelEncoder()
Final_df["Education_level"]=school_degree_encoder.fit_transform(Final_df["Education_level"])

In [6]:
from sklearn.preprocessing import LabelEncoder
business_domain_encoder= LabelEncoder()
Final_df["Work_domain"]=business_domain_encoder.fit_transform(Final_df["Work_domain"])

In [7]:
from sklearn.preprocessing import LabelEncoder
position_encoder= LabelEncoder()
Final_df["Position"]=position_encoder.fit_transform(Final_df["Position"])

In [8]:
from sklearn.preprocessing import LabelEncoder
Time_encoder= LabelEncoder()
Final_df["Time"]=Time_encoder.fit_transform(Final_df["Time"])

In [9]:
from sklearn.preprocessing import LabelEncoder
Day_encoder= LabelEncoder()
Final_df["Day"]=Day_encoder.fit_transform(Final_df["Day"])

In [10]:
from sklearn.preprocessing import LabelEncoder
Browser_encoder= LabelEncoder()
Final_df["Browser"]=Browser_encoder.fit_transform(Final_df["Browser"])

In [11]:
from sklearn.preprocessing import LabelEncoder
Location_encoder= LabelEncoder()
Final_df["Location"]=Location_encoder.fit_transform(Final_df["Location"])

In [12]:
from sklearn.preprocessing import LabelEncoder
device_encoder= LabelEncoder()
Final_df["Device"]=device_encoder.fit_transform(Final_df["Device"])


In [13]:
from sklearn.preprocessing import LabelEncoder
device_encoder= LabelEncoder()
Final_df["Expertise_level"]=device_encoder.fit_transform(Final_df["Expertise_level"])


#### Category
For the encoding of the category we see in the next cell that the gategories within the original dataset are redandant, and the same gategory is writen in diferent ways. Thats why we ned to filter it and only keep the unique category values 

In [14]:
Final_df["Category"]=Final_df["Category"].apply(lambda x: (((((x.lower())).replace("-", " ")).replace("based", "")).replace("attacks", "")).replace("attack", "").strip())
knowledge["Category"]=knowledge["category"].apply(lambda x: (((((x.lower())).replace("-", " ")).replace("based", "")).replace("attacks", "")).replace("attack", "").strip())

In [15]:
from sklearn.preprocessing import LabelEncoder
Category_encoder= LabelEncoder()
Final_df["Category"]=Category_encoder.fit_transform(Final_df["Category"])
knowledge["Category"]=Category_encoder.transform(knowledge["Category"])

# Sub algorithms

## Category Prediction

In [16]:
import pandas as pd
import os
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer#nltk.download('wordnet') 
from nltk.stem.wordnet import WordNetLemmatizer

In [17]:
nltk.download('stopwords')
nltk.download('wordnet')
##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
data = pd.read_excel("Keywords.xlsx",sheet_name="All data")

In [19]:
category = data['Category'].drop_duplicates()

In [20]:
category = np.array(category)

In [21]:
data["Category"]=data["Category"].apply(lambda x: (((((x.lower())).replace("-", " ")).replace("based", "")).replace("attacks", "")).replace("attack", "").strip())

In [22]:
categories = data['Category'].drop_duplicates()
categories = np.array(categories)
len(categories)
categories

array(['data breach', 'wireless', 'social engineering', 'network',
       'phishing', 'ransomware', 'man in the middle', 'ddos', 'malware',
       'not cyber'], dtype=object)

In [23]:
keyword = pd.read_excel("Keywords.xlsx",sheet_name="Keyword")
keyword.columns

Index(['Social engineering_keywords', 'Phishing_Keywords', 'Malware_Keywords',
       'Ransomware_Keywords', 'DDoS_Keywords', 'data_breach_keywords',
       'Man_in_the_middle _keywords', 'network_based_attack',
       'wireless_attack'],
      dtype='object')

In [24]:
keyword.columns

Index(['Social engineering_keywords', 'Phishing_Keywords', 'Malware_Keywords',
       'Ransomware_Keywords', 'DDoS_Keywords', 'data_breach_keywords',
       'Man_in_the_middle _keywords', 'network_based_attack',
       'wireless_attack'],
      dtype='object')

In [25]:
keyword.rename(columns={'Social engineering_keywords':'social engineering', 'Phishing_Keywords':'phishing',
                         'Malware_Keywords':'malware', 'Ransomware_Keywords':'ransomware',"DDoS_Keywords":'ddos',
                         'data_breach_keywords':'data breach','Man_in_the_middle _keywords':'man in the middle',
                         'network_based_attack':'network','wireless_attack':'wireless'},inplace=True)

In [26]:
def preproc(s):
  #corpus=[]
  #Remove punctuations
  text = re.sub('[^a-zA-Z]', ' ', s)
  #Convert to lowercase
  text = text.lower()
  #remove tags
  text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
  # remove special characters and digits
  text=re.sub("(\\d|\\W)+"," ",text)
  ##Convert to list from string
  text = text.split()
  ##Stemming
  ps=PorterStemmer()    #Lemmatisation
  lem = WordNetLemmatizer()
  text = [lem.lemmatize(word) for word in text if not word in  stop_words] 
  #text = " ".join(text)
  #corpus.append(text)
  return text

In [27]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [28]:
categories

array(['data breach', 'wireless', 'social engineering', 'network',
       'phishing', 'ransomware', 'man in the middle', 'ddos', 'malware',
       'not cyber'], dtype=object)

In [29]:
def category_prediction(input):
  votes= dict({'social engineering':0, 'phishing':0, 'malware':0,
       'ransomware':0, 'ddos':0, 'data breach':0,
       'man in the middle':0, 'network':0,
       'wireless':0})
  wordDict= dict.fromkeys(set(input), 0)
  tfDict = computeTF(wordDict, input)
  for w in input:
    for c in categories:
      if c =="not cyber":
        continue
      else:
        if w in keyword[c]:
          votes[c]+= tfDict[w]
  return max(votes)

In [30]:
inp = 'I recently visited a website, they asked me about my MAC adress, and my wifi network address, and i gave it to them, However i think that i had a security breach because my network trafic become very slow'
inp = preproc(inp)

In [31]:
pred = category_prediction(inp)
pred

'wireless'

In [32]:
def QA_algorithm(category, data):

  



  data_category= data[data["Category"]==category]
  indexes=data_category.index
  answer_found=False
  Final_answer=" "
  while (answer_found!=True) & (data_category.empty== False):
    stop=False
    data_categor=data_category.copy()
    sample=data_categor.sample()
    sample_index=sample.index[0]
    print(sample_index)

    q=sample["System_question1"].item()

    answ=[]
    i=1
    while i<=3:

        print(f'Chatbot {i}: {q} ')
        resp=input(f"User {i} ")
        data_categor=data_categor[(data_categor["System_question"+str(i)]==q) & (data_categor["answer"+str(i)]==resp)]
        
        if(i<3):
          q=data_categor["System_question"+str(i+1)]
        i=i+1
        #if resp=="yes":
          #resp=1
        #else:
          #resp=0
  
    Final_answer=data_categor['Final_answer'].values
    print(f"indice={data_categor['Final_answer'].index}")
    answer_found=True

    data_category=data_category.drop(index=sample_index)
  return Final_answer,data_category
    

In [33]:
prediction = category_prediction(inp)
category=Category_encoder.transform([prediction])[0]
final_answer,data_category=QA_algorithm(category,knowledge)
print(final_answer)

137
Chatbot 1: Did you notice any creation of new files without your knowledge? 
User 1 no
Chatbot 2: 140    Did you check the internet connection on the s...
141    Did you check the internet connection on the s...
142    Did you check the internet connection on the s...
143    Did you check the internet connection on the s...
Name: System_question2, dtype: object 
User 2 yes
Chatbot 3: 142    Did you change your credentials after connecti...
143    Did you change your credentials after connecti...
Name: System_question3, dtype: object 
User 3 yes
indice=Int64Index([143], dtype='int64')
['What makes SQL injection attacks even more menacing for businesses is the fact that these seemingly simple attacks can cost them not only hefty sums of money but may also lead to a lack of customer trust in the long run. And the fact that so many websites and web apps still face this vulnerability regularly reflects the seriousness of the issue.']


# Similarity Algorithm

For the Collaborative algorithm, we used the cosine similarity in order to compute the top n similar user to the current customer and provide the top rated answers for each user

In [59]:
def Similarity_based_recommendation(category,data,user_model,n):
  data=data[data["Category"]==category]
  original_data=data.copy()
  data=data[['ID', 'Age', 'Education_level', 'Work_domain', 'Position', 'Time',
       'Day', 'Browser', 'Location', 'Device', 'Expertise_level']].astype(np.int64)
  data=data.append(user_model)  
  dummy_data=pd.get_dummies(data,columns=['Age', 'Education_level', 'Work_domain', 'Position', 'Time',
       'Day', 'Browser', 'Location', 'Device', 'Expertise_level'])
  user_model=dummy_data.iloc[[-1]].astype(np.int64)
  data.drop_duplicates()
  dummy_data=dummy_data.drop_duplicates().astype(np.int64)
  #print(len(dummy_data.columns))
  #print(len(user_model.columns))
  #print(user_model["ID"])
  dummy_data=dummy_data[dummy_data["ID"]!=user_model["ID"].values[0]].astype(np.int64)
  user_model=user_model.drop("ID",axis=1)
  indexes=dummy_data.index.values
  # cosine formula 
  similarities=[]
  user_ids=[]
  similarity_indexes=[]
  for i in indexes:
    user_ids.append(dummy_data.loc[[i]]["ID"])
    dt=dummy_data.loc[[i]].drop("ID",axis=1)
    similarity_indexes.append(i)
    
    c=sum((dt.values[0]*user_model.values[0]))
    cosine = c / float((sum(dt.values[0])*sum(user_model.values[0]))**0.5)
    #print(f"similarity with user : {i} is {cosine}")
    similarities.append(cosine)
    #print(dummy_data.loc[[i]]["ID"])
    #print(cosine)
#########################################################################
  similarities=np.array(similarities)
  similarity_indexes=np.array(similarity_indexes)
  most_similar_indexes_ranged=similarities.argsort()[-n:][::-1]
  indexes_most=similarity_indexes[most_similar_indexes_ranged]
  print("most similar scenarios indexes")
  print(indexes_most)
  print("Most similar users")
  print(original_data.loc[indexes_most]["ID"].values)

  #print(indexes)

  
  answers=[]
  for i in indexes_most:
    
    answer=original_data.loc[[i]]["Final_answers"].values[0]
    answers.append(answer)

  return answers,indexes_most[0]


In [60]:
prediction = category_prediction(inp)

prediction

'wireless'

In [61]:
user_model=Final_df.loc[[8]][['ID', 'Age', 'Education_level', 'Work_domain', 'Position', 'Time',
       'Day', 'Browser', 'Location', 'Device', 'Expertise_level']]

user_model

,ID,Age,Education_level,Work_domain,Position,Time,Day,Browser,Location,Device,Expertise_level
8,2,1,8,4,3,3,3,3,0,4,0


In [62]:
category=Category_encoder.transform([prediction])[0]
similarities,i=Similarity_based_recommendation(category,Final_df,user_model,10)
similarities

most similar scenarios indexes
[ 10  73 299 655 166 657 236 248 149 294]
Most similar users
[  3  15  79 642  40 647  63  65  35  78]


['Few best practices to protect your web app from SQL injection attacks are 1. Proper filtration and sanitization of the user input 2. Implementations of least privilege principle 3. Improved code quality 4. Disable default accounts and credentials.',
 'Five common methods to test for SQLi vulnerabilities on your web app are Union Operator, Boolean, Error based, Out-of-band, Time delay techniques.',
 'There are several steps you can take to prevent our emails from getting hacked. 1.\xa0Consider the use of a Password Manager. Password managers store login details for all the websites that you use and logs you in automatically each time you return to a site. 2. Watch out for Suspicious Emails. 3.\xa0Use a VPN for Extra Security. 4.\xa0Activate Two-Factor Authentication. 5.\xa0Secure your Home Router and Wi-Fi',
 'The best way to prevent SQL Injections is to use safe programming functions that make SQL Injections impossible: parameterized queries (prepared statements) and stored procedure

In [63]:
i

10

In [38]:
Final_df.loc[[8]]

,ID,Age,Education_level,Work_domain,Position,Time,Day,Browser,Location,Device,Expertise_level,Final_answers,Category
8,2,1,8,4,3,3,3,3,0,4,0,Few practices can be considered to protect our...,8


In [39]:
Final_df.loc[[10 ]]


,ID,Age,Education_level,Work_domain,Position,Time,Day,Browser,Location,Device,Expertise_level,Final_answers,Category
10,3,1,8,15,3,3,2,3,1,4,0,Few best practices to protect your web app fro...,8


#Content based recommendation system

## Tree Model

In [50]:
!pip install -U scikit-multiflow

Requirement already up-to-date: scikit-multiflow in /usr/local/lib/python3.7/dist-packages (0.5.3)


In [51]:
# Imports
Tree_data=Final_df.copy()

from skmultiflow.data import DataStream
from skmultiflow.meta import AdaptiveRandomForestClassifier
# 
import pickle
with open('10Estimators_With_Overfitting_the_model.pickle', 'rb') as handle:
    ht = pickle.load(handle)

with open('RecommendationEncoder.pickle', 'rb') as handle:
    Final_answer_encoder = pickle.load(handle)
Tree_data["Final_answer"]=Final_answer_encoder.transform(Tree_data["Final_answers"])
Tree_data.drop(columns="Final_answers",inplace=True)

**At the end we will have a system that provides 3 recomendations:**
* one based on the QA
* one based on the similarity
* one based on the 'incremental' Tree model

=> the User will choose the most convinient one

If the user choose the recomendation provided by the incremental model then this is good. This is an indication that our model is capable of providing effective recommendations for the user.



If the user choose the recomendation provided by either the QA or similarity algorithm, then the model will be updated with the chosen recomendation, so that in the future if the user encounter similar problem, the model will predict the right answer because it already see it.

**Aron course min 31 -> 34** definition

min 37 admiting that this can work on any task

Reconciling modern machine learning practice
and the bias-variance trade-of

2019

JOURNAL: Proceedings of the National Academy of Sciences (Proceedings of the National Academy of Sciences of the United States of America) with 9.412 impact factor

# Main Algorithme 

Now for the global algorithm we will combine all the recomendations and demand the rating from the user

if the user rate one proposed solution more than 3.5 then the recomendation ends. Else the process will start again N times.

if the N recommendation step are ended and the user didn't rate one of them more than 3.5 then the system return 'sorry i can not give you the best recommendation'

# Scenario 1:
User found the best answer from the QA algo

In [64]:
user_model=Final_df.loc[[7]][['ID', 'Age', 'Education_level', 'Work_domain', 'Position', 'Time',
       'Day', 'Browser', 'Location', 'Device', 'Expertise_level','Category']]

In [65]:

inp = 'I received and email from my bank with a link inside it that told me to fill in a new password and login, However when i did that my password theft, and when ever i try to login, it tells me that i have a restrict access, What should i do'
inp = preproc(inp)
prediction = category_prediction(inp)
print(prediction)
#category=Category_encoder.transform(['data breach'])[0]
category=Category_encoder.transform([prediction])[0]
#category=Category_encoder.transform(['data breach'])[0]
data=Final_df.copy()

user_id=user_model["ID"].values[0]
um=user_model[['ID', 'Age', 'Education_level', 'Work_domain', 'Position', 'Time',
       'Day', 'Browser', 'Location', 'Device', 'Expertise_level']]

n=3

for i in range(n):
  answer_1,knowledge=QA_algorithm(category,knowledge)
  print(f"QA Algorithm recomendation is : {answer_1}")
  asnwer_2s=Similarity_based_recommendation(category,data,um,n)
  asnwer_2s,inde=Similarity_based_recommendation(category,data,um,n)
  data=data.drop(index=inde)
  if (len(data)>=n):
    answ2=asnwer_2s[i]
  else:
    answ2= "no more similar users in the dataset"
  
  
  print(f"Similarity recomendation is : {answ2}")
  answer_3 = Final_answer_encoder.inverse_transform(ht.predict(user_model.values))
  print(f"Tree model recommendation is {answer_3}")
  print(len(asnwer_2s))
  resp1=float (input(f"rate response 1 from 1 to 5 "))
  resp2=float (input(f"rate response 2 from 1 to 5 "))
  resp3=float (input(f"rate response 2 from 1 to 5 "))
  if (resp1>3.5) or (resp2>3.5)or (resp3>3.5):
    #save row for the user
    #or update the rating
      
      
      
    

    if resp1>3.5 :
      answ=answer_1
      r=resp1
    else :
      if resp2>3.5 :
        r=resp2
        answ=answ2
    ans=Final_answer_encoder.transform([answ])
    print(Final_answer_encoder.inverse_transform([ans]))
    ht=ht.partial_fit(user_model.values,ans)
    ht=ht.partial_fit(user_model.values,ans)
    ht=ht.partial_fit(user_model.values,ans)
    ht=ht.partial_fit(user_model.values,ans)
    ht=ht.partial_fit(user_model.values,ans)
    ht=ht.partial_fit(user_model.values,ans)
    dt=Final_df[(Final_df["ID"].values==user_id) & (Final_df["Category"].values==category)& (Final_df["Final_answers"].values==answ)]
    indu=um.index[0]
    print(indu)
    
    if dt.empty:
      #Store it in the database: i can not store it in the general database
      # you need to stor it in the user specific database 
      # because the general database contain the QA columns
      print('I will store it in the database')
      break
    else:
      dt['rating']=r
      break
  if i >= len(data):
    break
if (resp1<3.5) and (resp2<3.5):
  print('sorry i can not give you the best recommendation')

  
  


  


wireless
133
Chatbot 1: Did you experience the SQL injection attack recently? 
User 1 no
Chatbot 2: 132    Did you regularly do some backup of your data?
133    Did you regularly do some backup of your data?
Name: System_question2, dtype: object 
User 2 no
Chatbot 3: 132    Is the performance of computer dropping signif...
133    Is the performance of computer dropping signif...
Name: System_question3, dtype: object 
User 3 no
indice=Int64Index([132], dtype='int64')
QA Algorithm recomendation is : ['The defenses against an SQL injection attack are usually implemented at the code level, though administration best practices also play a part: Routinely update and patch all services, servers, and applications. Employ well-tested and well-implemented website code that doesn’t allow unexpected SQL commands. Implement prepared statements with parameterized queries. Build SQL statements using standard stored procedure programming constructs. Use whitelist input validation where applicable in S

In [53]:
.

In [ ]:
n